In [21]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from keras.applications.vgg16 import VGG16
from sklearn.metrics import confusion_matrix
from keras.applications.vgg16 import preprocess_input
from PIL import Image

In [22]:
# Define image size
img_size = (100, 100)
categories = ['cats', 'dogs']

# Change the path according to images location
dir = "./dataset/training_set/"


def load_dataset():
    # Define list for input and output data
    input = []
    output = []

    for type in categories:
        print(f'loading... category : {type}')
        path = os.path.join(dir, type)
        
        # Resize and flat every image in a category
        for img in os.listdir(path):
            img_path = os.path.join(path, img)
            img = Image.open(img_path)
            img = img.resize(img_size)
            img = np.array(img)
            img = preprocess_input(img)
            input.append(img)
            output.append(categories.index(type))

    input = np.array(input)
    output = np.array(output)
    return input, output

In [23]:
input, output = load_dataset()
print("Loaded")
# Split dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(
    input, output, test_size=0.2, random_state=42)

# Load the pre-trained VGG16 model without the classifier layer
model = VGG16(weights='imagenet', include_top=False,
              input_shape=(img_size[0], img_size[1], 3))

# Extract features from the training set
X_train_features = model.predict(X_train)
X_train_features = X_train_features.reshape(X_train_features.shape[0], -1)

# Define SVM classifier
svm = SVC(kernel="poly")
print("SVM Starts")

# Train the classifier
print("Start Fit")
svm.fit(X_train_features, Y_train)


# Extract features from the testing set
X_test_features = model.predict(X_test)
X_test_features = X_test_features.reshape(X_test_features.shape[0], -1)


# Make predictions on test set
Y_pred = svm.predict(X_test_features)
print("Predicted")

# Evaluate performance of SVM classifier
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)
confusion = confusion_matrix(Y_test, Y_pred)

print("Accuracy:", accuracy)
print("Classificatoin Report:", report)
print("Confusion matrix:\n", confusion)

loading... category : cats
loading... category : dogs
Loaded
200/200 [==============================] - 347s 2s/step
SVM Starts
Start Fit
50/50 [==============================] - 59s 1s/step
Predicted
Accuracy: 0.896875
Classificatoin Report:               precision    recall  f1-score   support

           0       0.84      0.99      0.91       804
           1       0.99      0.80      0.89       796

    accuracy                           0.90      1600
   macro avg       0.91      0.90      0.90      1600
weighted avg       0.91      0.90      0.90      1600

Confusion matrix:
 [[796   8]
 [157 639]]


In [24]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image

# initialise GUI
top = tk.Tk()
top.geometry('800x600')
top.title('CatsVSDogs Classification')
top.configure(background='#CDCDCD')
label = Label(top, background='#CDCDCD', font=('arial', 15, 'bold'))
sign_image = Label(top)


def classify(file_path):
    list = []
    img = Image.open(file_path)
    img = img.resize(img_size)
    img = np.array(img)
    img = preprocess_input(img)
    list.append(img)
    list = np.array(list)


    sample = model.predict(list)
    sample = sample.reshape(sample.shape[0], -1)

    
    pred = svm.predict(sample)

    sign = categories[pred[0]]
    print(sign)
    label.configure(foreground='#011638', text=sign)


def show_classify_button(file_path):
    classify_b = Button(top, text="Classify Image",
                        command=lambda: classify(file_path),
                        padx=10, pady=5)
    classify_b.configure(background='#364156', foreground='white',
                         font=('arial', 10, 'bold'))
    classify_b.place(relx=0.79, rely=0.46)


def upload_image():
    try:
        file_path = filedialog.askopenfilename()
        uploaded = Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width()/2.25),
                            (top.winfo_height()/2.25)))
        im = ImageTk.PhotoImage(uploaded)
        sign_image.configure(image=im)
        sign_image.image = im
        label.configure(text='')
        show_classify_button(file_path)
    except:
        pass


upload = Button(top, text="Upload an image",
                command=upload_image, padx=10, pady=5)
upload.configure(background='#364156', foreground='white',
                 font=('arial', 10, 'bold'))
upload.pack(side=BOTTOM, pady=50)
sign_image.pack(side=BOTTOM, expand=True)
label.pack(side=BOTTOM, expand=True)
heading = Label(top, text="CatsVSDogs Classification",
                pady=20, font=('arial', 20, 'bold'))
heading.configure(background='#CDCDCD', foreground='#364156')
heading.pack()
top.mainloop()


1/1 [==============================] - 0s 217ms/step
cats
1/1 [==============================] - 0s 69ms/step
cats
